In [1]:
import numpy as np
import pandas as pd
pop=pd.read_csv('data/state-population.csv')
area=pd.read_csv('data/state-areas.csv')
abbrevs=pd.read_csv('data/state-abbrevs.csv')
pop.head()
area.head()
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [2]:
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [3]:
area.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [4]:
# compute a relatively straightforward result: Rank US states by their 2010 population density.
# 1. merge state/region column of pop and abbreviation column of abbrevs. use how='outter' to make sure data is thrown away due to mismatched label

merged=pd.merge(pop,abbrevs, how='outter', left_on='state/region', right_on='abbreviation')


KeyError: 'outter'

In [5]:
# compute a relatively straightforward result: Rank US states by their 2010 population density.
# 1. merge state/region column of pop and abbreviation column of abbrevs. use how='outter' to make sure data is thrown away due to mismatched label

merged=pd.merge(pop,abbrevs, how='outer', left_on='state/region', right_on='abbreviation')

In [6]:
mergered

NameError: name 'mergered' is not defined

In [7]:
merged

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL
...,...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN,NaN
2540,USA,under18,2011,73902222.0,NaN,NaN
2541,USA,total,2011,311582564.0,NaN,NaN
2542,USA,under18,2012,73708179.0,NaN,NaN


In [8]:
merged=merged.drop('abbreviation',1) # drop duplicate infor
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [9]:
# check if any mismatch here
merged.isnull()

,state/region,ages,year,population,state
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
2539,False,False,False,False,True
2540,False,False,False,False,True
2541,False,False,False,False,True
2542,False,False,False,False,True


In [10]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [11]:
# there is some null information of population and state
# let's figure out which these are

merged[merged['population'].isnull()]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
2453,PR,under18,1993,NaN,NaN
2454,PR,under18,1992,NaN,NaN
2455,PR,total,1992,NaN,NaN
2456,PR,under18,1994,NaN,NaN
2457,PR,total,1994,NaN,NaN


In [12]:
# let's figure with state null 
merged[merged['state'].isnull()]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN
2540,USA,under18,2011,73902222.0,NaN
2541,USA,total,2011,311582564.0,NaN
2542,USA,under18,2012,73708179.0,NaN


In [13]:
# we can see state include USA.
# conclusion: PR state has null value for population.
#  we can see that there was no corresphonding entry in the abbrevs key. Let's figure out which regions lacks this match

merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [14]:
# we can quickly infer the issue: our pop data include entries for Puerto Rico(PR and the US as whole)
# while these entries do not appear in the state abbreviation key. We can fix these quickly by filling in approciate entries.

merged.loc[merged['state/region']=='PR','state']='Puerto Rio'
merged.loc[merged['state/region']=='USA','state']='United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [15]:
# no more null in the state columns
# now we can merge the result with the area data using a similar procedure.
# ex: we want to join on the state column in both
final=pd.merge(merged,area,on='state',how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [16]:
final['state'][final['area(sq.mi)'].isnull()].unique()

KeyError: 'area(sq.mi)'

In [17]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['Puerto Rio', 'United States'], dtype=object)

In [18]:
merged.loc[merged['state/region']=='PR','state']='Puerto Rico'
merged.loc[merged['state/region']=='USA','state']='United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [19]:
final=pd.merge(merged,area,on='state',how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [20]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [21]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [22]:
final.isnull().unique()

AttributeError: 'DataFrame' object has no attribute 'unique'

In [23]:
final.isna()

,state/region,ages,year,population,state,area (sq. mi)
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
2491,False,False,False,False,False,False
2492,False,False,False,False,False,False
2493,False,False,False,False,False,False
2494,False,False,False,False,False,False


In [24]:
final.isnull()

,state/region,ages,year,population,state,area (sq. mi)
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
2491,False,False,False,False,False,False
2492,False,False,False,False,False,False
2493,False,False,False,False,False,False
2494,False,False,False,False,False,False


In [25]:
final.isnull().unique()

AttributeError: 'DataFrame' object has no attribute 'unique'

In [26]:
# now we have data we need. To answer question of interest, let's first select the portion of data corresponding with the year 2010 and total population.
data2010=final.query("year==2010 & ages='total'")
data2010.head()

SyntaxError: Python keyword not valid identifier in numexpr query (<unknown>, line 1)

In [27]:
data2010=final.query("year==2010 & ages=='total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [28]:
# compute the population density and display it in order. 
# start by reindex our data on the state and then compute the result

data2010.set_index('state',inplace=True)
density=data2010['population']/data2010['area (sq. mi)']

In [29]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [30]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64